<a href="https://colab.research.google.com/github/Ayush-Singh-7/ML_Share_Doc/blob/master/APTOS_eye_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle

In [2]:
!apt-get install -y fuse-zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libzip4
The following NEW packages will be installed:
  fuse-zip libzip4
0 upgraded, 2 newly installed, 0 to remove and 32 not upgraded.
Need to get 65.6 kB of archives.
After this operation, 178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libzip4 amd64 1.1.2-1.1 [37.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fuse-zip amd64 0.4.4-1 [27.9 kB]
Fetched 65.6 kB in 2s (36.7 kB/s)
Selecting previously unselected package libzip4:amd64.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../libzip4_1.1.2-1.1_amd64.deb ...
Unpacking libzip4:amd64 (1.1.2-1.1) ...
Selecting previously unselect

In [0]:
import os

In [0]:
os.environ["KAGGLE_USERNAME"] = "ayushsingh7"
os.environ["KAGGLE_KEY"] = "606084e6b9fc1da0aa40da97a5deec92"


In [7]:
raw_data_dir = "input/raw"
!kaggle competitions download -c aptos2019-blindness-detection -p {raw_data_dir}

100% 8.01G/8.01G [02:26<00:00, 43.6MB/s]
100% 8.01G/8.01G [02:26<00:00, 58.7MB/s]
100% 1.50G/1.50G [00:20<00:00, 80.6MB/s]
100% 1.50G/1.50G [00:20<00:00, 77.0MB/s]
  0% 0.00/53.7k [00:00<?, ?B/s]
100% 53.7k/53.7k [00:00<00:00, 29.7MB/s]
  0% 0.00/24.5k [00:00<?, ?B/s]
100% 24.5k/24.5k [00:00<00:00, 21.3MB/s]
  0% 0.00/28.3k [00:00<?, ?B/s]
100% 28.3k/28.3k [00:00<00:00, 29.5MB/s]


In [17]:
%%time
input_dir = "/tmp/kaggle-data"
#!mkdir {input_dir}
!fuse-zip input/raw/train_images.zip {input_dir}


fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
CPU times: user 42.8 ms, sys: 9.17 ms, total: 52 ms
Wall time: 5.41 s


In [0]:
#!ls -ltr {input_dir}

In [21]:
%%time
input_dir_test = "/tmp/kaggle-data-test"
!mkdir {input_dir_test}
!fuse-zip input/raw/test_images.zip {input_dir_test}

CPU times: user 65.3 ms, sys: 15.3 ms, total: 80.6 ms
Wall time: 8.62 s


In [0]:
#!ls -ltr {input_dir_test}

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [0]:
base_dir = "../input/aptos2019-blindness-detection/"

train_csv = "input/raw/"+"train.csv"
train_dir = "/tmp/kaggle-data/" + "train_images/"

test_csv = "input/raw/" +"test.csv"
test_dir = "/tmp/kaggle-data-test/" + "test_images/"


IMG_SIZE = 224

SEED = 72

In [54]:
!ls -ltr {train_dir}

ls: cannot access '/tmp/kaggle-data/train_images/': No such file or directory


In [0]:
def df_train_test_split_preprocess(df):
    
    image_ids = df["id_code"].values.tolist()
    labels = df["diagnosis"].values.tolist()
    
    for i in range(len(image_ids)):
        imgname = image_ids[i]
        newname = str(imgname) + ".png"
        image_ids[i] = newname
    
    xtrain, xval, ytrain, yval = train_test_split(image_ids, labels, test_size = 0.15)
    
    df_train = pd.DataFrame({"id_code":xtrain, "diagnosis":ytrain})
    df_val = pd.DataFrame({"id_code":xval, "diagnosis":yval})
    
    df_train["diagnosis"] = df_train["diagnosis"].astype('str')
    df_val["diagnosis"] = df_val["diagnosis"].astype('str')
    
    print("Length of Training Data :",len(df_train))
    print("Length of Validation Data :",len(df_val))
    
    return df_train, df_val

In [0]:
import cv2 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight, shuffle

In [30]:
df = pd.read_csv(train_csv)
df_train, df_val = df_train_test_split_preprocess(df)

Length of Training Data : 3112
Length of Validation Data : 550


In [31]:
from keras.preprocessing.image import ImageDataGenerator

#   --- TO DO ----
#   zca_epsilon: epsilon for ZCA whitening. Default is 1e-6.
#   zca_whitening: Boolean. Apply ZCA whitening.
#
train_aug = ImageDataGenerator(rescale=1./255,
                               horizontal_flip = True,
                               zoom_range = 0.15,
                               vertical_flip = True,
                               shear_range=0.1,
                               rotation_range = 90
                               )

Using TensorFlow backend.


In [55]:
train_generator = train_aug.flow_from_dataframe(dataframe = df_train,
                                               directory = input_dir,
                                               x_col = "id_code",
                                               y_col = "diagnosis",
                                               batch_size = 16,
                                               target_size =  (IMG_SIZE, IMG_SIZE),
                                               #color_mode = 'grayscale',
                                               class_mode = "categorical")

Found 3112 validated image filenames belonging to 5 classes.


In [57]:
# Using same as for val
validation_generator = train_aug.flow_from_dataframe(dataframe = df_val,
                                                    directory = input_dir,
                                                    x_col = "id_code",
                                                    y_col = "diagnosis",
                                                    batch_size = 16, 
                                                    target_size = (IMG_SIZE, IMG_SIZE),
                                                    #color_mode = 'grayscale',
                                                    class_mode = "categorical")

Found 550 validated image filenames belonging to 5 classes.


In [0]:
test_df_orig = pd.read_csv(test_csv)

def process_test_df(test_df):
    test_ids = test_df["id_code"].values.tolist()
    for i in range(len(test_ids)):
        imgname = test_ids[i]
        newname = str(imgname) + ".png"
        test_ids[i] = newname
    test_df["id_code"] = test_ids
    return test_df

test_df = process_test_df(test_df_orig)

In [58]:
# No need to augment only rescale pixel values
test_aug = ImageDataGenerator(rescale = 1./255 )

test_generator = test_aug.flow_from_dataframe(dataframe = test_df, 
                                              directory = input_dir_test,
                                              x_col = "id_code",
                                              batch_size = 1,
                                              target_size =  (IMG_SIZE, IMG_SIZE), # to be changed as ???
                                              shuffle = False,
                                              class_mode = None)

Found 1928 validated image filenames.


In [0]:
#from keras.applications import ResNet50
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import keras
from keras.engine import Layer,InputSpec

In [39]:
import keras
from keras_applications.resnext import ResNeXt101

input_layer = Input(shape = (IMG_SIZE,IMG_SIZE,3))

base_model = ResNeXt101(weights = "imagenet" ,
                       include_top = True,
                       backend=keras.backend,
                       layers=keras.layers,
                       utils=keras.utils,
                       models=keras.models, 
                       input_tensor = input_layer)

#base_model.load_weights('../input/resnext/resnext101_tf_kernels.h5')

base_model.summary()













178176000/178173608 [==============================] - 14s 0us/step
Model: "resnext101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_________

In [40]:
#x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(base_model.layers[-2].output)
x = Dropout(0.35)(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

out = Dense(5, activation = 'softmax')(x)

model = Model(inputs = input_layer, outputs = out)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [41]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
optimizer = keras.optimizers.Adam(lr=2e-4)
#optimizer = keras.optimizers.Adam(lr=0.0005)
#
#optimizer = RAdam(lr=0.0005)
#optimizer = RAdam(lr=2e-4)
#es = EarlyStopping(monitor='val_loss', mode='min', patience = 9, restore_best_weights=True)
#rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience = 3, factor = 0.5, min_lr=1e-6)

es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=3,restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.2, 
                        patience=5, 
                        verbose=1, 
                        mode='auto', 
                        min_lr=1e-6)
#kappa_metrics = Metrics()
callback_list = [ rlrop ]

model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"]) 

In [0]:
history = model.fit_generator(generator = train_generator, 
                    steps_per_epoch = len(train_generator), 
                    epochs = 2, 
                    validation_data = validation_generator, 
                    validation_steps = len(validation_generator),
                    callbacks =  callback_list  )

Epoch 1/2
195/195 [==============================] - 626s 3s/step - loss: 0.8298 - acc: 0.6968 - val_loss: 0.7071 - val_acc: 0.7727
Epoch 2/2
 30/195 [===>..........................] - ETA: 5:52 - loss: 0.6966 - acc: 0.7396